<a href="https://colab.research.google.com/github/AravDharnikota/Civora.AI/blob/main/event_key_idea_extraction_model_qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### imports

In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.1/126.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/90/e5/b22697903982284fe284568fb2663a2196694a8eee637f5cf4ccfe435a38/auto_gptq-0.7.1.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto-gptq from https://files.pythonhosted.org/packages/90/e5/b22697903982284fe284568fb2663a2196694a8eee637f5cf4ccfe435a38/auto_gptq-0.7.1.tar.gz has inconsistent version: expected '0.7.1', but metadata has '0.7.1+cu1251'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/34/71/c3e73cf17681f6ff4754ef8f4cb8b67af3def230fc8711eac1250bbd78d5/auto_gptq-0.7.0.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto-gptq from https://files.pythonhosted.org/packages/34/71/c3e73cf17681f6ff4754ef8f4cb8b67af3

In [2]:
# resolving "No inf checks were recorded for this optimizer." issue
!pip uninstall torch -y
!pip install torch

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 16.7 MB/s eta 0:00:00
  

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

### Load model

In [4]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

### Load tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

#### Prompt Engineering

In [6]:
intstructions_string = f"""You extract the SINGLE PRIMARY EVENT this article is about and return EXACTLY ONE strict JSON object (no array, no prose, no code fences). Output VALID JSON (RFC 8259): double quotes, no trailing commas, null/true/false in lowercase. Do NOT add extra keys.

FIELDS
- who: array of [name, role]                # actors of THIS event only; canonical full names (e.g., "Donald Trump")
- to_whom: array of [name, role[]            # direct target/recipient (empty if none; may include city/org)
- action_lemma: lowercase verb              # e.g., "post", "sign", "sue", "arrest", "protest", "announce", "launch", "file"
- action_phrase: ≤6 words copied from the evidence sentence (e.g., "posted meme", "signed executive order")
- when: [start:null, end:null, grain:"second|minute|hour|day|week|month|year|unknown", text]
  • If only a weekday/date phrase (e.g., "Friday", "next week"), put it in when.text, set grain="day", and keep start=end=null.
  • Unless an explicit clock time appears in the article, keep start=end=null.
- where: [name:<city>, country:<country or "" if unknown>]
  • Prefer city,country; do NOT use building names (e.g., not "City Hall"). Leave country "" if unknown.
- summary: 1–3 sentence neutral summary of the event (no new facts beyond the article)
- evidence: ONE exact sentence from the article that states the action (verbatim, single sentence only)
- confidence: number 0–1
- extras: object (free-form key/values; not a string), e.g., ["platform":"social media","media_type":"meme"]

RULES
- Output exactly ONE JSON object and nothing else.
- "who" MUST include only actors that appear in the evidence sentence OR the sentence immediately before it.
- Use canonical full names (no titles). If the article later says “Trump” but earlier gives “Donald Trump,” use “Donald Trump.”
- If multiple concrete events are described, choose the most supported by the headline/lead and least speculative.
- Do not invent facts. Use "" or null when unknown.
"""

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Prepare Model for Training

In [8]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [9]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 688,128 || all params: 1,544,402,432 || trainable%: 0.0446


### Preparing Training Dataset

In [10]:
# load dataset
from datasets import load_dataset

TRAIN_PATH = "/content/drive/MyDrive/Civora AI/train.jsonl"
DEV_PATH   = "/content/drive/MyDrive/Civora AI/dev.jsonl"

ds = load_dataset(
    "json",
    data_files={"train": TRAIN_PATH, "eval": DEV_PATH},
)

print(ds)                 # DatasetDict with train/eval
print(ds["train"][0])     # first sample (should have "messages")

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 421
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 105
    })
})
{'messages': [{'role': 'system', 'content': 'You extract the SINGLE PRIMARY EVENT this article is about and return EXACTLY ONE strict JSON object (no array, no prose, no code fences). Follow this schema:\n\nFIELDS\n- who: array of {name, role}               # actors of THIS event only; use canonical full names (e.g., "Donald Trump")\n- to_whom: array of {name, role}           # direct target/recipient (empty if none; can include city/org)\n- action_lemma: lowercase verb             # e.g., "post", "sign", "sue", "arrest", "protest", "announce", "launch", "file"\n- action_phrase: ≤6 words from the evidence sentence (e.g., "posted meme", "signed executive order")\n- when: {start:null, end:null, grain:"second|minute|hour|day|week|month|year|unknown", text}\n  • If only a weekday/date phrase (e.g., "Friday", "nex

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer

MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

ds = load_dataset("json", data_files={"train": TRAIN_PATH, "eval": DEV_PATH})

def to_text(example):
    txt = tok.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False   # we include the assistant label in training
    )
    return {"text": txt}

ds_text = ds.map(to_text, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/421 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

In [12]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Fine-tuning Model

In [13]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "article_key_idea_extraction",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [14]:
!pip -q install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 9.6 MB/s eta 0:00:00


In [15]:
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

# 1) Load your JSONL (each line has {"text": "..."} )
ds = load_dataset(
    "json",
    data_files={
        "train": "/content/drive/MyDrive/Civora AI/train.jsonl",
        "eval":  "/content/drive/MyDrive/Civora AI/dev.jsonl",
    }
)

# 2) LoRA config (safe defaults)
peft_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]   # common for Qwen; you can add k_proj/o_proj later
)

# 3) SFT (training) config — note: use max_length (not max_seq_length)
sft_cfg = SFTConfig(
    output_dir="/content/drive/MyDrive/Civora AI/ckpts",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=20,
    eval_strategy="steps",       # <- correct name in TRL 0.23
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    fp16=True,                   # T4 = fp16; set bf16=True if on A100/H100
    packing=False,               # keep simple; you can try packing=True later
    max_length=3072,             # <- replaces max_seq_length; remove if it errors and rely on model_max_length
    # For Qwen2.5 chat models, set EOS to the chat end token so training stops correctly:
    eos_token="<|im_end|>",
    # (Optional) pass model.from_pretrained kwargs here:
    model_init_kwargs={
        "torch_dtype": "float16",        # or leave to auto
        "attn_implementation": "eager"   # safer if FlashAttn not available
    }
)

# 4) Build trainer — pass model *name* (SFTTrainer will load tokenizer internally)
trainer = SFTTrainer(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    args=sft_cfg,
    train_dataset=ds["train"],
    eval_dataset=ds["eval"],
    peft_config=peft_cfg,
)

# 5) Train
trainer.model.config.use_cache = False
trainer.train()
trainer.save_model()
trainer.model.config.use_cache = True

`torch_dtype` is deprecated! Use `dtype` instead!


Tokenizing train dataset:   0%|          | 0/421 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/421 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adharnikota (adharnikota-civora-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


### Push model to hub

In [21]:
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [22]:
hf_name = 'AravD' # your hf username or org name
model_id = hf_name + "/" + "article_key_idea_extraction"

In [23]:
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ptxq8jayk/adapter_model.safetensors:  38%|###7      | 1.04MB / 2.76MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

...

  ...ents.1758086280.6f0f3d8f744b.7392.0:  58%|#####8    | 3.66kB / 6.26kB            

  ...ents.1758086354.6f0f3d8f744b.7392.1:  58%|#####8    | 3.66kB / 6.26kB            

  ...vents.1758086952.f087d4ec303e.843.0:  58%|#####8    | 5.05kB / 8.65kB            

  ...e/Civora AI/ckpts/training_args.bin:  58%|#####8    | 3.67kB / 6.29kB            

CommitInfo(commit_url='https://huggingface.co/AravD/ckpts/commit/02994027170df9106d1fe18e54223d3bc1a58740', commit_message='AravD/article_key_idea_extraction', commit_description='', oid='02994027170df9106d1fe18e54223d3bc1a58740', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AravD/ckpts', endpoint='https://huggingface.co', repo_type='model', repo_id='AravD/ckpts'), pr_revision=None, pr_num=None)